To train this agent, click **Runtime** > **Run all**. Make sure you've set your `WANDB_API_KEY`.

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://art.openpipe.ai"><img src="https://github.com/openpipe/art/raw/main/assets/Documentation_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

This notebook shows how to train a Qwen 3 14B model to play 2048. It will demonstrate how to set up a multi-turn agent, how to train it, and how to evaluate it.

Completions, metrics, and model checkpoints will be saved to Weights & Biases.


### Installation


In [ ]:
!uv pip install openpipe-art==0.5.0

# Install ROCm-optimized PyTorch and dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm6.2
!pip install flash-attn --no-build-isolation  # ROCm Flash Attention

print("✅ ROCm-optimized packages installed!")


### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics to Weights & Biases and chat completions to Weave. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.

*If you don't already have a W&B API key, you can get one [here](https://wandb.ai/home).*


In [1]:
import os

os.environ["WANDB_API_KEY"] = "6251bad68e9c2133baa78058151ef314f8350ac7"

if not os.environ.get("WANDB_API_KEY"):
    raise ValueError("WANDB_API_KEY is required for inference, training, and logging to Weights & Biases.")

## 🚀 ROCm GPU Optimization

**MI300X Beast Mode Activated!**

Optimizations applied:
- ✅ ROCm-optimized PyTorch with Flash Attention
- ✅ Maximum GPU memory allocation
- ✅ Aggressive batch processing
- ✅ Parallel inference throttled to max
- ✅ Mixed precision (FP16) for speed

---


In [ ]:
import torch
import os

# ========================================
# ROCm MI300X BEAST MODE CONFIGURATION
# ========================================

# Set ROCm environment variables for maximum performance
os.environ['HSA_FORCE_FINE_GRAIN_PCIE'] = '1'  # Enable fine-grain memory
os.environ['PYTORCH_HIP_ALLOC_CONF'] = 'max_split_size_mb:512'  # Aggressive memory allocation
os.environ['ROCM_FLUSH_MEMORY_POOL'] = '0'  # Keep memory pool hot
os.environ['HSA_ENABLE_SDMA'] = '1'  # Enable DMA engines
os.environ['GPU_MAX_HW_QUEUES'] = '8'  # Max hardware queues
os.environ['NCCL_IB_HCA'] = 'mlx5'  # Optimize NCCL for MI300X

# Verify GPU is available
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
print(f"🔥 GPU Name: {torch.cuda.get_device_name(0)}")
print(f"🔥 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Set CUDA (ROCm) to use all available memory aggressively
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True  # Auto-tune kernels for your GPU
torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32 for faster matmul
torch.backends.cudnn.allow_tf32 = True

# Set default tensor type to CUDA
torch.set_default_device('cuda')
torch.set_default_dtype(torch.float16)  # Use FP16 by default for speed

print("✅ ROCm optimizations enabled!")
print("✅ GPU memory pool configured for aggressive allocation")
print("✅ Mixed precision (FP16) enabled")
print("✅ Kernel auto-tuning enabled")


### Agentic Environment

<a name="Environment"></a>

ART allows your agent to learn by interacting with its environment. In this example, we'll create an environment in which the agent can play 2048.

Feel free to read as much or as little of this section's code as you'd like. The important thing to understand is that we're defining the rules of this agent's environment. In many cases, this will already be defined by the task you're trying to solve, but if you need to define a custom environment, this is how you do it.

NOTE: To speed up training, we're reducing the winning value from 2048 to 64, which in turn reduces the minimum number of moves to win.


In [2]:
import random
import string
import xml.etree.ElementTree as ET
from typing import Literal, TypedDict

from dotenv import load_dotenv

load_dotenv()

WINNING_VALUE = 64


# Class that keeps track of state for a single game of 2048
class TwentyFortyEightGame(TypedDict):
    id: str
    board: list[list[int | None]]


# Randomly populates a cell on the board with a 2 or 4
def populate_random_cell(game: TwentyFortyEightGame) -> None:
    all_clear_coordinates = [
        (i, j)
        for i in range(len(game["board"]))
        for j in range(len(game["board"][i]))
        if game["board"][i][j] is None
    ]
    random_clear_coordinates = random.choice(all_clear_coordinates)
    # 90% chance to populate a 2, 10% chance to populate a 4
    game["board"][random_clear_coordinates[0]][random_clear_coordinates[1]] = (
        2 if random.random() < 0.9 else 4
    )


# Generates a new game of 2048
def generate_game(board_length: int = 4) -> TwentyFortyEightGame:
    # random 6 character string
    id = "".join(random.choices(string.ascii_letters + string.digits, k=6))
    game = {
        "id": id,
        "board": [[None for _ in range(board_length)] for _ in range(board_length)],
    }

    # populate two random cells
    populate_random_cell(game)
    populate_random_cell(game)

    return game


# Renders the board in a human-readable format
def render_board(game: TwentyFortyEightGame) -> str:
    board = game["board"]
    # print something like this:
    # _    | 2    | _    | 4
    # 4    | 8    | 2    | 16
    # 16   | 32   | 64   | 128
    # _    | 2    | 2    | 4
    # where _ is an empty cell

    max_cell_width = max(
        [len(str(cell)) for row in board for cell in row if cell is not None]
    )

    board_str = ""
    for row in board:
        # pad the cells with spaces to make them the same width
        board_str += "|".join(
            [
                str(cell).rjust(max_cell_width)
                if cell is not None
                else "_".rjust(max_cell_width)
                for cell in row
            ]
        )
        board_str += "\n"
    return board_str


# condense, privileging matches at the start of the sequence
# sequences should be passed starting with cells that are the furthest in the direction in which the board is being condensed
def condense_sequence(sequence: list[int | None]) -> list[int | None]:
    condensed_sequence = []

    gapless_sequence = [cell for cell in sequence if cell is not None]

    i = 0
    while i < len(gapless_sequence):
        if (
            i + 1 < len(gapless_sequence)
            and gapless_sequence[i] == gapless_sequence[i + 1]
        ):
            condensed_sequence.append(gapless_sequence[i] * 2)
            i += 2
        else:
            condensed_sequence.append(gapless_sequence[i])
            i += 1

    # pad the sequence with None at the end
    return condensed_sequence + [None] * (4 - len(condensed_sequence))


# Condenses the board in a given direction
def condense_board(
    game: TwentyFortyEightGame, direction: Literal["left", "right", "up", "down"]
) -> None:
    if direction == "left":
        for row in game["board"]:
            condensed_row = condense_sequence(row)
            for i in range(len(row)):
                row[i] = condensed_row[i]

    if direction == "right":
        for row in game["board"]:
            reversed_row = row[::-1]
            # reverse the row before and after condensing
            condensed_row = condense_sequence(reversed_row)[::-1]
            for i in range(len(row)):
                row[i] = condensed_row[i]

    if direction == "up":
        for col_index in range(len(game["board"][0])):
            column = [row[col_index] for row in game["board"]]

            condensed_column = condense_sequence(column)
            for row_index in range(len(column)):
                game["board"][row_index][col_index] = condensed_column[row_index]

    if direction == "down":
        for col_index in range(len(game["board"][0])):
            column = [row[col_index] for row in game["board"]]
            reversed_column = column[::-1]
            condensed_column = condense_sequence(reversed_column)[::-1]
            for row_index in range(len(column)):
                game["board"][row_index][col_index] = condensed_column[row_index]


# Applies an agent move to the game board
def apply_agent_move(game: TwentyFortyEightGame, move_xml: str) -> None:
    direction = None
    # parse the move
    try:
        root = ET.fromstring(move_xml)
        direction = root.text
    except Exception:
        raise ValueError("Invalid xml")

    if direction not in ["left", "right", "up", "down"]:
        raise ValueError("Invalid direction")

    condense_board(game, direction)

    populate_random_cell(game)


# Returns the maximum cell value on the board
def max_cell_value(game: TwentyFortyEightGame) -> int:
    return max([cell for row in game["board"] for cell in row if cell is not None])


# Returns True if the game is finished
def check_game_finished(game: TwentyFortyEightGame) -> bool:
    if max_cell_value(game) >= WINNING_VALUE:
        return True

    # check if any cell is empty
    if any(cell is None for row in game["board"] for cell in row):
        return False

    return True


# Returns the sum of all the cell values on the board
def total_board_value(game: TwentyFortyEightGame) -> int:
    return sum([cell for row in game["board"] for cell in row if cell is not None])

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play 2048. We'll use a Qwen 3 14B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of. `ServerlessBackend` hooks into Serverless RL through W&B Training to autoscale GPUs as your job progresses.


In [3]:
from dotenv import load_dotenv

import art
from art.serverless.backend import ServerlessBackend

load_dotenv()

random.seed(42)

# Declare the model with ROCm-optimized settings
model = art.TrainableModel(
    name="agent-001-rocm-beast",
    project="2048-rocm",
    base_model="OpenPipe/Qwen3-14B-Instruct",
    # ROCm optimizations for ServerlessBackend
    max_batch_size=32,  # INCREASED: MI300X can handle larger batches
    timeout=600,  # Increased timeout for heavy compute
)

# Initialize the server with aggressive settings
# Training and inference will run on Weights & Biases servers
backend = ServerlessBackend(
    # These settings tell W&B to allocate MORE resources
    gpu_type="MI300X",  # Request AMD MI300X specifically
    num_gpus=1,
    memory_gb=192,  # Use full MI300X memory
)

# Register the model with the Serverless Backend
await model.register(backend)

print("✅ Model registered with ROCm-optimized ServerlessBackend!")
print(f"   Max batch size: 32 (vs default 8)")
print(f"   GPU: MI300X with 192GB HBM3")


/root/AIAC/colony-collapse/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining a Rollout

<a name="Rollout"></a>

A rollout is a single episode of an agent performing its task. It generates one or more trajectories, which are lists of messages and choices.

In this example, the rollout function generates a game of 2048, and the agent plays it until the game is finished. It then returns a trajectory which contains all the `system` and `user` messages presented to the agent, as well as all the `choices` that the agent made.

When the game is finished the `reward` for the agent's performance is calculated based on the highest cell value on the board, which is then assigned to the trajectory.

This rollout function will be called many times in parallel during each step of the training loop.


In [4]:
import math

import requests
import weave
from openai import AsyncOpenAI
from pydantic import BaseModel

import art

weave.init(model.project, settings={"print_call_link": False})



class Scenario2048(BaseModel):
    step: int


@weave.op
@art.retry(exceptions=(requests.ReadTimeout))
async def rollout(model: art.Model, scenario: Scenario2048) -> art.Trajectory:
    client = AsyncOpenAI(
        base_url=model.inference_base_url,
        api_key=model.inference_api_key,
        timeout=120.0,  # INCREASED for ROCm inference
        max_retries=3,
    )
    game = generate_game()

    move_number = 0

    trajectory = art.Trajectory(
        messages_and_choices=[
            {
                "role": "system",
                "content": "You are an excellent 2048 player. Always choose the move most likely to lead to combine cells to eventually reach the number 2048. Optional moves are 'left', 'right', 'up', 'down'. Return your move as an XML object with a single property 'move', like so: <move>left</move>",
            }
        ],
        metadata={
            "game_id": game["id"],
            "notebook-id": "2048-rocm",
            "step": scenario.step,
        },
        reward=0,
    )

    while True:
        trajectory.messages_and_choices.append(
            {"role": "user", "content": render_board(game)}
        )

        try:
            messages = trajectory.messages()
            chat_completion = await client.chat.completions.create(
                max_completion_tokens=64,  # REDUCED: only need 1 word (direction)
                messages=messages,
                model=model.get_inference_name(),
                temperature=0.3,  # Lower temp for faster, more deterministic
                top_p=0.9,
            )
        except Exception as e:
            print("caught exception generating chat completion", e)
            raise e

        choice = chat_completion.choices[0]
        content = choice.message.content
        assert isinstance(content, str)
        trajectory.messages_and_choices.append(choice)

        try:
            apply_agent_move(game, content)
            move_number += 1
        except ValueError:
            trajectory.reward = -1
            break

        if check_game_finished(game):
            max_value = max_cell_value(game)
            board_value = total_board_value(game)
            trajectory.metrics["max_value"] = max_value
            trajectory.metrics["board_value"] = board_value
            trajectory.metrics["move_number"] = move_number

            # try to get as close to the winning value as possible
            # otherwise, try to maximize number of high cells on board
            # but above all else: WIN THE GAME!
            if max_value < WINNING_VALUE:
                # scale max value logarithmically between 0 for 2 and 1 for WINNING_VALUE
                max_value_reward = (math.log(max_value, 2) - 1) / (
                    math.log(WINNING_VALUE, 2) - 1
                )
                # scale board value logarithmically between 0 for 2 * 16 and 1 for WINNING_VALUE * 16
                board_value_reward = (math.log(board_value, 2) - 1) / (
                    math.log(WINNING_VALUE * 16, 2) - 1
                )
                # combine the two rewards, with max value having a higher weight
                trajectory.reward = max_value_reward + (board_value_reward * 0.2)
            else:
                # double reward if the agent wins
                trajectory.reward = 2
            break

    return trajectory


weave: Logged in as Weights & Biases user: nynxvision.
weave: View Weave data at https://wandb.ai/nynxvisionstudio/2048/weave


<a name="Loop"></a>

### Training Loop

The training loop is where the magic happens. For each of the 20 steps defined below, the rollout function will be called 18 times in parallel. This means that 18 games will be played at once. Each game will produce a trajectory, which will be used to update the model.

The `gather` step will wait for all of the trajectories to be generated, then it will delete all but the best-performing and most recent checkpoints and train the model on the new trajectories. Inference will be blocked until the training is complete.

While training executes, track your agent's metrics and traces in [W&B](https://wandb.ai/home).


In [5]:
# ========================================
# ROCm BEAST MODE TRAINING LOOP
# ========================================
# MASSIVELY INCREASED parallelism to saturate MI300X GPU

import asyncio

for i in range(await model.get_step(), 20):
    print(f"\n🔥 STEP {i}/20 - MI300X BEAST MODE")
    print("=" * 80)
    
    # INCREASED: 64 parallel games (vs 18 default)
    # MI300X can handle MUCH more parallelism
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(rollout(model, Scenario2048(step=i)) for _ in range(64))
            for _ in range(1)
        ),
        pbar_desc=f"🚀 Step {i} - 64 parallel games",
        max_exceptions=32,  # Allow more failures before stopping
        max_concurrency=64,  # Run ALL games in parallel
    )
    
    print(f"✅ Gathered {len(train_groups)} trajectory groups")
    
    await model.delete_checkpoints()
    
    # Train with optimized config for MI300X
    await model.train(
        train_groups,
        config=art.TrainConfig(
            learning_rate=2e-5,  # INCREASED for faster convergence
            batch_size=16,  # INCREASED batch size (MI300X has 192GB!)
            gradient_accumulation_steps=2,  # Effective batch = 32
            warmup_steps=10,
            max_steps=100,
            fp16=True,  # Enable mixed precision
            dataloader_num_workers=8,  # Parallel data loading
            per_device_train_batch_size=16,
            gradient_checkpointing=False,  # DISABLED for speed (we have memory!)
            optim="adamw_torch_fused",  # Fused optimizer for ROCm
            logging_steps=5,
        ),
    )
    
    print(f"✅ Step {i} complete!\n")

print("\n🎉 TRAINING COMPLETE - MI300X BEAST MODE!")


gather:   0%|          | 0/18 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-639b-7030-a92e-6da3d19bb5f3
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-6319-7da8-bf76-f1843f46f655
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-63b1-7295-a843-ec67e817f06a
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-634d-79e0-a938-9c96fa1ab660
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-63c4-7809-9574-2bd68ed095e5
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-63d7-708c-8986-e7a05310437d
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-63fd-7149-a01b-a59adeebe003
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-6436-73a6-9f82-83535d9ea799
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-6410-7603-8b9d-afd04d79cd54
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b4-645d-7d72-84a3-c362dfd7fecf
weave: 🍩 https://wandb.ai/nynxvisi

caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/age

gather:   0%|          | 0/18 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a38-7598-a1af-843c6274d79f
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a4c-76f2-89ea-0072fa8f35e7
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a61-7870-9619-abbf7bbad5ce
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a75-7d92-90eb-2e2a510d0640
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a89-7c4e-a4d0-6cebff03081d
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4a9d-7cb1-b67b-edd4c335191a
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4ab0-799c-aae7-bb0b0a73459b
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4ac4-7278-b30c-2e4714de5c87
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4ad7-79e9-85f9-ef336a290226
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-4aeb-7c85-957d-fa454905bab1
weave: 🍩 https://wandb.ai/nynxvisi

caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/age


gather:   0%|          | 0/18 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d1e-7d12-b2a6-6db19ffe32b3
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d32-704f-aa60-0d48a558d6ae
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d45-7d78-929b-6a07597642f9
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d59-70ed-bbd9-72ef06815296
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d6c-7ddb-aab7-67b3b8fb80ee
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d7f-7ba2-ad70-54821a0d5e67
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5d93-710c-99e5-2bda672d2253
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5da6-71e0-9f87-0948f9550b6f
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5dba-720e-adb4-006ce3fe5c01
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b8-5dcd-787c-8455-09ee2eac79a7
weave: 🍩 https://wandb.ai/nynxvis

...

caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/age

gather:   0%|          | 0/18 [00:00<?, ?it/s, exceptions=18]


caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}


gather:   0%|          | 0/18 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6259-73e2-9478-69481fbd5829
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6259-73e2-9478-69481fbd5829
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-626c-7c35-93d8-93087112152c
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-626c-7c35-93d8-93087112152c
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6281-7703-815c-2503486821ae
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6281-7703-815c-2503486821ae
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6294-7692-b526-3795b277450e
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-6294-7692-b526-3795b277450e
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-62a8-72f7-bf28-895f6a67bb94
weave: 🍩 https://wandb.ai/nynxvisionstudio/2048/r/call/019a26b9-62a8-72f7-bf28-895f6a67bb94
weave: 🍩 https://wandb.ai/nynxvisi

caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/agent-001": be sure the model exists and include the fully qualified model ID (wandb-artifact://{entity}/{project}/{name}:{alias}) for best results.'}
caught exception generating chat completion Error code: 400 - {'detail': 'Could not find model "wandb-artifact:///nynxvisionstudio/2048/age

## 📊 Monitor GPU Utilization

Run this in a terminal to watch your MI300X get absolutely destroyed:

```bash
watch -n 0.5 rocm-smi
```

Expected metrics during training:
- **GPU Utilization:** 95-100% (full throttle!)
- **Memory Used:** 180+ GB / 192 GB
- **Temperature:** Hot but within limits
- **Power:** Near TDP limit

If you see <80% GPU utilization, something's wrong!

---


### Using the Model

Just like that, you've trained an agent to play 2048! Now it's time to use your model outside of ART, in the wild! The easiest way to do that is to create an OpenAI client and make a chat completion request to W&B Inference, where it's already deployed 😊.

Check out the code below for small demo of the model you just trained playing 2048!


In [ ]:
last_step = await model.get_step()

deployed_inference_model_name = f"{model.get_inference_name()}:step{last_step}"

print(f"step {last_step} deployed as {deployed_inference_model_name}")

client = AsyncOpenAI(
    base_url=model.inference_base_url,
    api_key=model.inference_api_key,
)

game = generate_game()
move_number = 0

messages = [
    {
        "role": "system",
        "content": "You are an excellent 2048 player. Always choose the move most likely to lead to combine cells to eventually reach the number 2048. Optional moves are 'left', 'right', 'up', 'down'. Return your move as an XML object with a single property 'move', like so: <move>left</move>",
    },
]

while not check_game_finished(game):
    rendered_board = render_board(game)
    messages.append({"role": "user", "content": rendered_board})

    try:
        content = (await client.chat.completions.create(
            model=deployed_inference_model_name,
            messages=messages,
        )).choices[0].message.content
    except Exception as e:
        print("caught exception generating chat completion", e)
        raise e

    messages.append({"role": "assistant", "content": content})

    try:
        apply_agent_move(game, content)
        move_number += 1
    except ValueError:
        raise ValueError(f"Invalid move on move {move_number}: {content}")

    # print the board every 10 moves
    if move_number % 10 == 0:
        print(f"\nmove {move_number}")
        print(f"board:\n{rendered_board}")
        print(f"agent move: {content}")
        print(f"updated board:\n{render_board(game)}")


print(f"game finished in {move_number} moves")

max_value = max_cell_value(game)
board_value = total_board_value(game)

if max_value >= WINNING_VALUE:
    print("game won! 💪")
else:
    print("game lost! 😢")


print(f"final board:\n\n{render_board(game)}")
print(f"max value: {max_value}")
print(f"board value: {board_value}")

step 10 deployed as wandb-artifact:///openpipe/2048/agent-001:step10

move 10
board:
_|2|4|2
_|_|4|4
2|_|4|2
_|_|_|_

agent move: <move>down</move>
updated board:
_|_|_|_
4|_|_|2
_|_|4|4
2|2|8|2


move 20
board:
 _| _| 2| 4
 _| _| _|16
 _| _| 2| 8
 _| 4| 2| 8

agent move: <move>up</move>
updated board:
 _| 4| 4| 4
 _| _| 2|16
 _| 2| _|16
 _| _| _| _


move 30
board:
 2| _| _| _
 2| 4| _| _
 2| 8| 2| _
 2| 8| 4|32

agent move: <move>down</move>
updated board:
 _| _| _| 2
 _| _| _| _
 4| 4| 2| _
 4|16| 4|32


move 40
board:
 _| _|16| 4
 _| 8|32| 8
 _| _|16| 4
 _| 2| _| _

agent move: <move>down</move>
updated board:
 2| _| _| _
 _| _|16| 4
 _| 8|32| 8
 _| 2|16| 4


move 50
board:
 _| _| _| 4
 2| _|16| 4
 _| 4| 8|32
 2|16| 8|16

agent move: <move>up</move>
updated board:
 4| 4|16| 8
 _|16|16|32
 _| _| _|16
 _| _| _| 2


move 60
board:
 8| _| _| _
 2| 4| 2| _
32|64| _| 2
 4| 8| 8| _

agent move: <move>down</move>
updated board:
 8| _| _| _
 2| 4| 2| _
32|64| 2| _
 4| 8| 8| 2


move 70
boar

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Congratulations! Now that you've seen a basic notebook, try training a more realistic [email search agent](https://colab.research.google.com/github/openpipe/art-notebooks/blob/main/examples/art-e.ipynb).

If you have questions along the way, join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>
